In [112]:
import pandas as pd
import numpy as np
import requests

def get_ranking(ranking_type='headphones'):
    url = f'https://crinacle.com/rankings/{ranking_type}/'

    # Pretend we are a browser to avoid being blocked
    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }
    r = requests.get(url, headers=header)
    dat = pd.read_html(r.text)[0]
    
    # Deal with both missing prices and weirdly formatted prices
    dat = dat.query('Pricesort != 99999')
    dat.loc[:,'Price (MSRP)'] = pd.to_numeric(dat['Price (MSRP)'], errors='coerce')
    dat.dropna(subset=['Price (MSRP)'], inplace=True)
    
    # Add the bang for buck variable
    ranks = dat['Rank'].unique()
    rank_number = np.linspace(100, 1, len(ranks))
    rank_number_lut = dict(zip(ranks, rank_number))
    dat.loc[:,'Rank_number'] = dat['Rank'].replace(rank_number_lut)
    dat.loc[:,'bang_buck'] = (dat['Rank_number'] / dat['Price (MSRP)'])
    
    # Recast Rank as a Categorical, needed for proper order of ranks in plots
    all_possible_ranks = list(chain.from_iterable([[s+'+', s, s+'-'] for s in list(string.ascii_uppercase[0:6])]))
    dat.loc[:,'Rank'] = pd.Categorical(dat['Rank'], categories = all_possible_ranks)

    # Save the rankings that where generated from the data, for future reference
    dat.attrs['rank_number_lut'] = rank_number_lut
    return dat

def select_products(dat, min_price = 50, max_price = 100, min_rank='C+', sort_to='bang_buck', open_closed=None, model_search=None):
    subset = (
        dat
          .query(f'{max_price} >= `Price (MSRP)` >= {min_price}')                   # Price Range
          .query('Rank_number >= %f' % dat.attrs['rank_number_lut'][min_rank])      # Minimum Rank
          .sort_values(sort_to, ascending=False)                                    # Sort according to bang for buck
    )
    if open_closed is not None:
        subset = subset[subset['Fit/Cup Type'].str.contains(open_closed)]
    if model_search is not None:
        subset = subset[subset['Model'].str.contains(model_search)]
    return subset

headphone_data = get_ranking('headphones')

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/opt/conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [114]:
select_products(headphone_data, min_rank='C', 
                min_price=20, max_price=100).head(10)

,Rank,Value Rating,Model,Price (MSRP),Signature,Comments,Tone Grade,Technical Grade,Driver Type,Fit/Cup Type,Based on,Note weight,Ranksort,Tonesort,Techsort,Pricesort,Rank_number,bang_buck
81,B-,★★★,Koss KSC75,20.0,Bass-rolled neutral,Non-existent sub-bass and poor resolution. But...,A,C,Dynamic,Open Clip-on,Personal unit,NaN,132,21,111,20,50.500000,2.525000
51,B,★★★,Shure SRH440,100.0,Harman,Closest thing to Harman thus far. Comes with a...,S-,C+,Dynamic,Closed Circumaural,Connect IT demo unit,NaN,98,7,91,100,57.571429,0.575714
107,C+,★,1MORE MK801,80.0,Neutral with bass boost,"Smooth, generally pleasing tonality throughout...",A-,C-,Dynamic,Closed Circumaural,Jaben SG demo unit,NaN,164,31,133,80,43.428571,0.542857
88,B-,★,Audio Technica ATH-M40x,100.0,Neutral with bass boost,Well-balanced tonality with disappointing deta...,A-,C,Dynamic,Closed Circumaural,Zeppelin & Co demo unit,NaN,142,31,111,100,50.500000,0.505000
90,B-,★,E-MU Walnut,100.0,"""Balanced""",Tonality is somewhat off with peaks and dips h...,B-,B-,Dynamic,Closed Circumaural,Zeppelin & Co demo unit,NaN,146,73,73,100,50.500000,0.505000


In [84]:
import altair as alt
#from vega_datasets import data

plot_data = select_products(headphone_data, min_rank='E', 
                min_price=50, max_price=400)
alt.Chart(plot_data).mark_circle(size=60).encode(
    x='bang_buck',
    y=alt.Y('Rank', sort=alt.Sort(list(headphone_data['Rank'].cat.categories))), 
    color='Price (MSRP)',
    tooltip=['Model', 'Rank', 'bang_buck', 'Price (MSRP)']
).interactive()

alt.Chart(...)

In [121]:
plot_data = select_products(headphone_data, min_rank='E', 
                min_price=0, max_price=1000, open_closed='Open', model_search='Sennheiser').sort_values('bang_buck', ascending=False).head(20)

alt.Chart(plot_data).mark_bar().encode(
    x='bang_buck',
    y=alt.Y("Model", sort=list(plot_data.sort_values('bang_buck', ascending=False)['Model'])),
    color='Rank',
    tooltip=['Model', 'Rank', 'Tone Grade', 'Technical Grade', 'Price (MSRP)']
)

alt.Chart(...)

In [127]:
plot_data = select_products(headphone_data, min_rank='E', 
                min_price=0, max_price=1000, open_closed='Open', model_search='Hifi').sort_values('bang_buck', ascending=False).head(20)

alt.Chart(plot_data).mark_bar().encode(
    x='bang_buck',
    y=alt.Y("Model", sort=list(plot_data.sort_values('bang_buck', ascending=False)['Model'])),
    color='Rank',
    tooltip=['Model', 'Rank', 'Tone Grade', 'Technical Grade', 'Price (MSRP)']
)

alt.Chart(...)

In [134]:
plot_data = select_products(headphone_data, min_rank='B+', 
                min_price=100, max_price=10000, open_closed='Open').sort_values('bang_buck', ascending=False).head(20)

alt.Chart(plot_data).mark_bar().encode(
    x='bang_buck',
    y=alt.Y("Model", sort=list(plot_data.sort_values('bang_buck', ascending=False)['Model'])),
    color='Rank',
    tooltip=['Model', 'Rank', 'Tone Grade', 'Technical Grade', 'Price (MSRP)']
)

alt.Chart(...)